# Declare all required dependencies and import data that will required to build model upon.

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

campaign = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv')


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37064,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
37065,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
37066,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
37067,44,technician,married,professional.course,no,no,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


# Prepare data

In [60]:
from pandas.core.indexing import convert_missing_indexer
training, testing = train_test_split(campaign, test_size=0.2, random_state=42)

X_train = training.drop('y', axis=1)
y_train = training['y'].map({'yes': 1, 'no': 0})

X_test = testing.drop('y', axis=1)
y_test = testing['y'].map({'yes': 1, 'no': 0})

numerics = [ 'cons.conf.idx']
nonnumeric = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

ctMinMax = ColumnTransformer(
    transformers=[
        ('confidentMax', MinMaxScaler(), numerics),
        ('time', OneHotEncoder(drop='first'), nonnumeric)])

In [61]:
# Train the Decision Tree model
dtc = DecisionTreeClassifier(max_depth=5, random_state=42)

# Create a pipeline for the model
Model = Pipeline(steps=[
    ('preprocessor', ctMinMax),
    ('classifier', dtc)
])

# Fit the model on the training data
Model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('confidentMax',
                                                  MinMaxScaler(),
                                                  ['cons.conf.idx']),
                                                 ('time',
                                                  OneHotEncoder(drop='first'),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'month', 'day_of_week',
                                                   'poutcome'])])),
                ('classifier',
                 DecisionTreeClassifier(max_depth=5, random_state=42))])

In [62]:
y_train_pred = Model.predict(X_train)
check = 0;
for x in y_train_pred:
  if x == 1:
    check+=1
print(check)

1087


In [63]:
# Evaluate the model on the training data
print("Training accuracy:", accuracy_score(y_train, y_train_pred))
print("Training classification report:\n", classification_report(y_train, y_train_pred))
print("Training confusion matrix:\n", confusion_matrix(y_train, y_train_pred))

# Make predictions on the test data
y_test_pred = Model.predict(X_test)

# Evaluate the model on the test data
print("\nTest accuracy:", accuracy_score(y_test, y_test_pred))
print("Test classification report:\n", classification_report(y_test, y_test_pred))
print("Test confusion matrix:\n", confusion_matrix(y_test, y_test_pred))
print("First 10 test predictions:", y_test_pred[:10])

Training accuracy: 0.9024447816557073
Training classification report:
               precision    recall  f1-score   support

           0       0.91      0.99      0.95     26333
           1       0.70      0.23      0.34      3322

    accuracy                           0.90     29655
   macro avg       0.80      0.61      0.65     29655
weighted avg       0.89      0.90      0.88     29655

Training confusion matrix:
 [[26004   329]
 [ 2564   758]]

Test accuracy: 0.8938494739681684
Test classification report:
               precision    recall  f1-score   support

           0       0.90      0.99      0.94      6528
           1       0.69      0.21      0.32       886

    accuracy                           0.89      7414
   macro avg       0.79      0.60      0.63      7414
weighted avg       0.88      0.89      0.87      7414

Test confusion matrix:
 [[6445   83]
 [ 704  182]]
First 10 test predictions: [0 0 0 0 0 0 0 0 0 0]
